In [1]:
# setup notebook
# imports
import os
import numpy as np
import random

import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import output_notebook
output_notebook() # set default; alternative is output_file()


# notebook formatting
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# fix RISE scollbar missing
from traitlets.config.manager import BaseJSONConfigManager
path = "C:\\Users\\chris\\.jupyter\\nbconfig"
cm = BaseJSONConfigManager(config_dir=path)

cm.update("livereveal", {
              "scroll": True,
});

# also check jupyter Cell menu/Toggle Scrolling


Loading BokehJS ...

# Meaning of Bokeh
- Japanese word **“bokeh”** used in photography to describe *blurring of the out-of-focus parts of an image.*<br><br>
<img src="https://p2.piqsels.com/preview/926/415/630/wheat-ear-dry-harvest.jpg" height=200 style="float:center"> <br><br>
<strong>[James Drury example - Flickr](https://www.flickr.com/photos/james_drury/15923166238/)</strong><br><br>
- How do you pronounce this crazy word?
  - bouquet
  - bok-ah
  - **both are fine**

# Bokeh
- [bokeh homepage](https://bokeh.pydata.org/en/latest/index.html)<br>
- python library 
- focused on interactive visualization
- targets browsers for presentation <br>
- goals: elegant, concise, designed for large/streaming data


# Two modes
- client based
- server

- Bokeh uses Python to create high level objects (plots, subplots, lines, etc).
- ...and then renders everything in Javascript
- ...for the browser to display

<strong>workflow:</strong><br>
data --> python --> Bokeh --> Bokeh ColumnDataSource--> BokehJS --> Javascript -->Browser --> your eyes


# Bokeh Code

# five ways to interact using Bokeh
- Configuring the toolbar
- Selecting data points
- Adding hover actions
- Linking subplots and selections
- Highlighting data using the legend<br>

## also can interact with ipywidgets

# Example 1 - basic charts and interactions

In [2]:
from bokeh.plotting import figure, output_notebook, show

x = [x for x in range(0, 11)]
y = [9, 8, 7, 5, 4, 6, 8, 3, 2, 0, 1]

chart = figure(title="simple line chart", 
               x_axis_label="x axis!", y_axis_label="y_axis",
               toolbar_location="right")

chart.xaxis.axis_label_text_font_size = "18pt"
chart.yaxis.axis_label_text_font_size = "18pt"

chart.line(x, y, line_width=2)

show(chart)

# review interactive tools

In [3]:
# adding tools and annotations

from bokeh.models import LassoSelectTool
from bokeh.models import BoxAnnotation

chart.add_tools(LassoSelectTool())
low_box = BoxAnnotation(top=4.2, bottom=1.6, fill_alpha=0.1, fill_color='green')
chart.add_layout(low_box)

show(chart)

In [4]:
# Example 2 - interaction with ipywidgets

In [5]:
# simple interactivity example - using ipywidgets
# credit:  https://github.com/bokeh/bokeh/blob/1.3.4/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb

from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

# define evenly spaced data to plot trig functions
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

graph_function = figure(title="example - simple trig plot", plot_height=300, plot_width=900, y_range=(-10,10),
           background_fill_color='#efefef')
r = graph_function.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

def update(function, freq=10, Amplitude=2, phi=0):
    if   function == "sin": func = np.sin
    elif function == "cos": func = np.cos
    elif function == "tan": func = np.tan
    r.data_source.data['y'] = Amplitude * func(freq * x + phi)
    push_notebook()

interact(update, function=["sin", "cos", "tan"], freq=(0,500), Amplitude=(1,20), phi=(0, 20, 0.5))
show(graph_function, notebook_handle=True);  # handle updates existing plot, only needed in jupyter notebook (not jupyterlab)


Loading BokehJS ...

interactive(children=(Dropdown(description='function', options=('sin', 'cos', 'tan'), value='sin'), IntSlider(…

# Introducing Bokeh ColumnDataSource object<br>

- Bokeh class
- Bokeh can plot directly from Pandas DataFrames AND
- ColumnDataSource helps maps names of columns to sequences or arrays <br><br>


    from bokeh.models import ColumnDataSource

    data = {"x": [1,2,3,4], "y": [11, 22, 33, 44], labels=["alpha", "bravo", "charlie", "delta"]}

    source = ColumnDataSource(data,  color=Colorblind)
<br>

### why use this?
- easier to share data + selections between plots and subplots<br>
- efficient graphing of streaming - bokeh only sends new data to plots<br>
- offloads work to the browser (e.g. colormapping)<br><br>

- color examples:  https://docs.bokeh.org/en/latest/docs/reference/palettes.html

# Example 3 - Saving Data from A Chart

*WIP*

# Example 4 - Interactive, linked brushing

In [6]:
from random import random

from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show

# output_file("callback.html")

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
s2 = ColumnDataSource(data=dict(x=[], y=[]))
s3 = ColumnDataSource(data=dict(x=[], y=[]))

p1 = figure(plot_width=400, plot_height=400, tools=["lasso_select", "reset"], title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.4)

p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="save", title="Watch First Graph Here")
p2.circle('x', 'y', source=s2, color="forestgreen", alpha=0.6)

p3 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="save", title="Watch Second Mirror Graph Here")
p3.circle('x', 'y', source=s3, color="firebrick", alpha=0.3)


s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2, s3=s3), code="""
        var inds = cb_obj.indices;
        var d1 = s1.data;
        
        // data for 2nd graph
        var d2 = s2.data;     
        d2['x'] = []
        d2['y'] = []     
 
        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        
        // data for 3rd graph
        var d3 = s3.data;
        d3['x'] = []
        d3['y'] = []
        d3['x'] = d1['x'].filter(x => !d2['x'].includes(x));
        d3['y'] = d1['y'].filter(y => !d2['y'].includes(y));
        
        s2.change.emit();
        s3.change.emit();
    """)
)

# layout = column(row(p1, p2), p3)
# show(layout)

grid = gridplot([[p1, p2], [None, p3]])
show(grid)

# Server Based Plotting

### why use a server?
- data >> local machine or notebook<br><br>

-- create deployable apps<br><br>

- need to sync between python and the end user:<br>
-- respond to browser events with computations or queries using python<br>
-- automatically push updates the UI browser<br>
-- streaming (periodic, timeout, and async callbacks)<br><br>

- Bokeh's reactive client-server model can trigger server-side code
- server supports automatic downsampling

# [link to Example 4 - working with Streaming Data](http://localhost:8888/notebooks/notebooks/03_bokeh_with_streaming_data.ipynb)

# bokeh resources

[bokeh cheat sheet](https://datacamp-community-prod.s3.amazonaws.com/f9511cf4-abb9-4f52-9663-ea93b29ee4b7)